In [8]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
from qiskit import Aer
from tqdm.notebook import tqdm

import sys
sys.path.insert(0, '../../src/')
from neuralnetwork import *
from analysis import *
from data_encoders import *
from parametrizations import *

#%matplotlib notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
backend = Aer.get_backend('qasm_simulator')
n_samples = 4
n_features = 3
n_targets = 1

x = np.linspace(0, 2, n_samples)
y = x**2
y = y - np.min(y)
y = y/np.max(y)
y = np.pi *y
y = y.reshape(-1,1)

X = np.zeros((n_samples, n_features))
X[:,0] = x
X[:,1] = x**2
X[:,2] = x**3
#X[:,3] = x**4



X = X - np.min(X, axis=0).reshape(1,-1)
X = X/np.max(X, axis=0).reshape(1,-1)
X = np.pi * X

print(X)
print(y)

[[0.000 0.000 0.000]
 [1.047 0.349 0.116]
 [2.094 1.396 0.931]
 [3.142 3.142 3.142]]
[[0.000]
 [0.349]
 [1.396]
 [3.142]]


In [18]:
reps = 4
np.random.seed(42)
model = ParallelModel(n_features=n_features,
                      n_targets=n_targets,
                      reps=reps,
                      backend=backend, 
                      shots=100000)

optimizer = Adam()
optimizer.initialize(model.theta.shape)

In [ ]:
for i in tqdm(range(100)):
    grad = [model.gradient(X, y)]
    grad = optimizer(grad)
    model.theta += -0.05*grad[0]
    y_pred = model.predict(X)
    mse = np.mean((y_pred - y)**2)
    loss = model.loss(X,y)
    #print(model.theta)
    print(f"{loss=:.3f}, {mse=:.3f}, {y_pred.flatten()=}")

  0%|          | 0/100 [00:00<?, ?it/s]

loss=0.758, mse=1.777, y_pred.flatten()=array([1.465, 2.174, 1.607, 1.883])
loss=0.730, mse=1.752, y_pred.flatten()=array([1.416, 2.151, 1.523, 1.824])
loss=0.704, mse=1.731, y_pred.flatten()=array([1.370, 2.116, 1.450, 1.756])
loss=0.682, mse=1.681, y_pred.flatten()=array([1.298, 2.069, 1.362, 1.700])
loss=0.663, mse=1.612, y_pred.flatten()=array([1.226, 2.006, 1.293, 1.663])
loss=0.642, mse=1.535, y_pred.flatten()=array([1.146, 1.940, 1.235, 1.635])
loss=0.631, mse=1.453, y_pred.flatten()=array([1.071, 1.877, 1.199, 1.628])
loss=0.629, mse=1.389, y_pred.flatten()=array([1.024, 1.828, 1.193, 1.631])
loss=0.622, mse=1.327, y_pred.flatten()=array([0.973, 1.791, 1.186, 1.646])
loss=0.624, mse=1.276, y_pred.flatten()=array([0.934, 1.767, 1.208, 1.664])
loss=0.621, mse=1.218, y_pred.flatten()=array([0.887, 1.735, 1.237, 1.679])
loss=0.620, mse=1.157, y_pred.flatten()=array([0.829, 1.702, 1.278, 1.694])
loss=0.616, mse=1.100, y_pred.flatten()=array([0.766, 1.664, 1.316, 1.701])
loss=0.609, 

In [ ]:
reps = 2
np.random.seed(42)
model = ParallelModel(n_features=n_features,
                      n_targets=n_targets,
                      reps=reps,
                      backend=backend, 
                      shots=100000)

backend = Aer.get_backend('qasm_simulator')
n_samples = 2
n_features = 2
n_targets = 1

x = np.linspace(0, 2, n_samples)

X = np.zeros((n_samples, n_features))
X[:,0] = x
X[:,1] = x**2



X = X - np.min(X, axis=0).reshape(1,-1)
X = X/np.max(X, axis=0).reshape(1,-1)
X = np.pi * X

y = model.predict(X)
print(y)

loss = model.loss(X,y)
print(loss)

In [ ]:
data = np.array([[0,0], [1,1], [2,2], [3,3]])
encoder = ParallelEncoder()


features = qk.QuantumRegister(2, name="features")
ancilla = qk.QuantumRegister(2, name="ancilla_f")
registers = [features, ancilla]
circuit = qk.QuantumCircuit(*registers)

circuit = encoder(circuit, features, ancilla, data)

circuit.draw()